In [1]:
import numpy as np
from keras import layers
from keras.layers import Input, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D
from keras.layers import AveragePooling2D, MaxPooling2D, Dropout, GlobalMaxPooling2D, GlobalAveragePooling2D
from keras.models import Model
from keras.preprocessing import image
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras.applications.imagenet_utils import preprocess_input
import pydot
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model

import keras.backend as K
K.set_image_data_format('channels_last')
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow

import h5py
import cv2
import os

%matplotlib inline

/home/apurvnit/anaconda3/lib/python3.5/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
def load_dataset():
    train = h5py.File('dataset/train_happy.h5', "r")
    X_train_orig = np.array(train["train_set_x"][:]) # your train set features
    Y_train_orig = np.array(train["train_set_y"][:]) # your train set labels

    test_dataset = h5py.File('dataset/test_happy.h5', "r")
    X_test_orig = np.array(test_dataset["test_set_x"][:]) # your test set features
    Y_test_orig = np.array(test_dataset["test_set_y"][:]) # your test set labels

    classes = np.array(test_dataset["list_classes"][:]) # the list of classes
    
    Y_train = Y_train_orig.reshape((1, Y_train_orig.shape[0]))
    Y_test = Y_test_orig.reshape((1, Y_test_orig.shape[0]))
    
    return X_train_orig, Y_train, X_test_orig, Y_test, classes



In [3]:
X_train_o, Y_train_o, X_test_o, Y_test_o, classes = load_dataset()

In [4]:
X_train = X_train_o/255
X_test = X_test_o/255

Y_train = Y_train_o.T
Y_test = Y_test_o.T

print(Y_train.shape, X_train.shape)

(600, 1) (600, 64, 64, 3)


In [5]:
def HappyModel(input_shape):
   
    X_input = Input(input_shape)

    X = ZeroPadding2D((3, 3))(X_input)

    # CONV -> BN -> RELU Block applied to X
    X = Conv2D(32, (7, 7), strides = (1, 1), name = 'conv0')(X)
    X = BatchNormalization(axis = 3, name = 'bn0')(X)
    X = Activation('relu')(X)

    # MAXPOOL
    X = MaxPooling2D((2, 2), name='max_pool')(X)

    # FLATTEN X (means convert it to a vector) + FULLYCONNECTED
    X = Flatten()(X)
    X = Dense(1, activation='sigmoid', name='fc')(X)
    model = Model(inputs = X_input, outputs = X, name='HappyModel')
    
    return model

In [6]:
model = HappyModel((64,64,3))

In [7]:
model.compile(loss='binary_crossentropy',optimizer="adam",metrics=['accuracy'])

In [10]:
model.fit(x=X_train, y=Y_train, epochs=100, batch_size=25)

Epoch 1/100
600/600 [==============================] - 6s 11ms/step - loss: 0.0475 - acc: 0.9833
Epoch 2/100
600/600 [==============================] - 7s 11ms/step - loss: 0.0969 - acc: 0.9733
Epoch 3/100
600/600 [==============================] - 7s 12ms/step - loss: 0.0293 - acc: 0.9917
Epoch 4/100
600/600 [==============================] - 9s 15ms/step - loss: 0.0104 - acc: 0.9983
Epoch 5/100
600/600 [==============================] - 8s 14ms/step - loss: 0.0255 - acc: 0.9883
Epoch 6/100
600/600 [==============================] - 13s 22ms/step - loss: 0.0153 - acc: 0.9917
Epoch 7/100
600/600 [==============================] - 8s 13ms/step - loss: 0.0158 - acc: 0.9983
Epoch 8/100
600/600 [==============================] - 7s 11ms/step - loss: 0.0069 - acc: 0.9983
Epoch 9/100
600/600 [==============================] - 6s 10ms/step - loss: 0.0046 - acc: 0.9967
Epoch 10/100
600/600 [==============================] - 6s 10ms/step - loss: 0.0137 - acc: 0.9933
Epoch 11/100
600/600 [======

600/600 [==============================] - 7s 12ms/step - loss: 0.0157 - acc: 0.9967
Epoch 84/100
600/600 [==============================] - 7s 11ms/step - loss: 6.0142e-05 - acc: 1.0000
Epoch 85/100
600/600 [==============================] - 7s 11ms/step - loss: 8.3739e-04 - acc: 1.0000
Epoch 86/100
600/600 [==============================] - 7s 11ms/step - loss: 0.0011 - acc: 1.0000
Epoch 87/100
600/600 [==============================] - 7s 11ms/step - loss: 0.0046 - acc: 0.9983
Epoch 88/100
600/600 [==============================] - 7s 12ms/step - loss: 6.2377e-05 - acc: 1.0000
Epoch 89/100
600/600 [==============================] - 7s 11ms/step - loss: 7.3919e-05 - acc: 1.0000
Epoch 90/100
600/600 [==============================] - 7s 12ms/step - loss: 3.4819e-05 - acc: 1.0000
Epoch 91/100
600/600 [==============================] - 7s 12ms/step - loss: 2.1763e-05 - acc: 1.0000
Epoch 92/100
600/600 [==============================] - 7s 11ms/step - loss: 2.6774e-05 - acc: 1.0000
Epoch

In [20]:
preds = model.evaluate(x=X_train, y=Y_train)
print()
print ("Loss = " + str(preds[0]))
print ("Test Accuracy = " + str(preds[1]*100) + " %")

600/600 [==============================] - 4s 6ms/step

Loss = 1.8450210786795653e-05
Test Accuracy = 100.0 %


In [26]:
import time

def save_model(file):
    print("Creating Model At: ",file) 
    start_time = time.time()
    
    json_model = model.to_json()
    model.save_weights("weights.h5")
    
    with open(file, "w") as json_file:
        json_file.write(json_model)
    
    end_time = time.time()
    total_time = end_time-start_time
    print("Model Created: ",total_time, " seconds")


In [27]:
save_model('model.json')

Creating Model At:  model.json
Model Created:  0.03480887413024902  seconds


In [28]:
plot_model(model, to_file='model.png', show_shapes= True, show_layer_names = True)

In [29]:
def test(model, img_path):
    img = cv2.imread(img_path)
    img.resize(64,64,3)
    print(img.shape)
    k = model.predict(x=np.array([img]))[0][0]
    print(k)
    if k>0.66:
        print("Happy in the picture")
    elif k<0.33:
        print("sad in the picture")
    else:
        print("normal")

In [30]:
test(model,'test/pp.jpeg')
test(model, 'test/my_image.jpg')

(64, 64, 3)
1.0
Happy in the picture
(64, 64, 3)
1.0
Happy in the picture
